In [4]:
import sys; sys.path.insert(0, '../')
from PlexSim.plexsim.models import *
n = 128
g = nx.grid_graph([n, n])
# g = nx.grid_2d_graph(n, n, periodic = 1)

settings = dict(graph = g,\
                updateType = 'async',\
#                 sampleSize = .1,\
                alpha = 4,\
                mu = .001,\
                nu = .1,\
                kappa = .001,\
                beta = .5,\
                threshold = .01,\
                p = .1,\
               t = .5)

models = dict(
    Potts = Potts(agentStates = np.arange(5).tolist(),\
          **settings),\
#     Ising = Ising(\
#           **settings),\
#     Bornholdt = Bornholdt(\
#               **settings),\
#     RBN = RBN(**settings),\
#     SIRS = SIRS(\
#         **settings),\
#    CCA = CCA(\
#         agentStates = np.arange(0,4).tolist(),\
#         **settings),\
#    Percolation = Percolation(\
#                **settings),\
   
)

# models.get("Bornholdt").sampleSize = 1
if m := models.get("SIRS"):
    m.states = 0
    m.states[m.sampleNodes(1)[0,0]] = 1
print("starting sims")
T = 1000
import time
start = time.time()
results = np.zeros((len(models), T, g.number_of_nodes()))
for idx, m in enumerate(models.values()):
    results[idx] = m.simulate(T)
results = results.reshape(-1, T, n, n)

# print(results.shape)
# results = results[-1, T,:, :, 0]
print(time.time() - start)

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable as mal
%matplotlib widget
ts = IntSlider(min = 0, max = T - 1, value = 0)

columns, rows = divmod(len(models), 2)
fig, ax = plt.subplots(rows +1, columns +1, figsize = (5,5))

fig.subplots_adjust(wspace = .12, hspace = .0001)
#setup figure
hs  = []
pad = .01
size = '5%'
for idx, ( (modelName, m), axi) in enumerate(zip(models.items(), ax.flat)):
    div = mal(axi)
    cbr = div.append_axes('bottom', size = size, pad = pad)
#     div = mal(cbr)
#     cbr = div.append_axes('right', size = size, pad = pad)
    AS = m.agentStates
    NS = len(AS) 
    cmap = mpl.colors.LinearSegmentedColormap.from_list(\
          None, plt.cm.viridis(np.linspace(0, 255, NS,\
                                          dtype = int)), NS)
    h = axi.imshow(results[idx, 0], cmap = cmap, vmin = AS[0],\
                  vmax = AS[-1])
    axi.set_title(modelName)
    axi.axis("off")
    tmp = fig.colorbar(h, cax = cbr, ticks = AS,\
                       orientation = 'horizontal')
    if modelName == 'SIRS':
        tmp.set_ticklabels("Sus. Inf. Rec.".split())
    hs.append(h)
mainax = fig.add_subplot(111, frameon = 0,\
                        xticks = [], yticks = [])
[axi.axis('off') for axi in ax.flat[rows * columns:]]
mainax.set_xlabel(f'T = 0')
@interact(t = ts)
def update(t):
    for idx, h in enumerate(hs):
        h.set_data(results[idx, t])
#         h.autoscale()
    mainax.set_xlabel(f'T = {t}')
    fig.canvas.flush_events()
    fig.canvas.draw()
    
fig.show()

# next(iter(models.values())).magnetize()

starting sims
7.300043821334839


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

In [2]:
# %load_ext line_profiler
# %reload_ext line_profiler
# %load_ext memory_profiler
# m = models.get("Ising")
# # %prun -s line m.simulate(100)
# # from model_profile import test
# %prun -s nfl m.sampleNodes(100)
# # divmod(len(models), 2)

In [3]:
from matplotlib.animation import FuncAnimation as fan
#an = fan(fig, update, frames = np.arange(0, int(T*1), 10), interval = 100,\

#         blit = True) 
#an.save('PlexSim_banner.mp4', writer = 'ffmpeg')